# Elixir: debugging

### `IO.inspect/2`

`IO.inspect(item, opts \\ [])` returns the item argument passed to it without affecting the behavior of the original code. Therefore you can “spy” on values almost anywhere in your code without altering the result, making it very helpful inside of a pipeline.

In [1]:
(1..10)
|> IO.inspect
|> Enum.map(fn x -> x*2 end)
|> IO.inspect
|> Enum.sum
|> IO.inspect

`IO.inspect/2` also provides the ability to decorate the output with a `label` option. The label will be printed before the inspected item.

In [2]:
[1, 2, 3]
|> IO.inspect(label: "before")
|> Enum.map(&(&1 * 2))
|> IO.inspect(label: "after")
|> Enum.sum

before: [1, 2, 3]
after: [2, 4, 6]


12

### `IEx.pry/0` and `IEx.break!/2`

While `IO.inspect/2` is static, Elixir’s interactive shell provides more dynamic ways to interact with debugged code.

The first one is with `IEx.pry/0` which we can use instead of `IO.inspect binding()`.

__NOTE: this example returns ArgumentError: cannot invoke def/2 outside module. Defining a module around this definition does not resolve the issue. Running the code directly n IEx does not resolve it either. Requires further examination.__

In [3]:
def thisfun(a,b,c) do
  require IEx; IEx.pry
  ...
  end

ArgumentError: 1

Once the code above is executed inside an `iex` session, IEx will ask if we want to pry into the current code. If accepted, we will be able to access all variables, as well as imports and aliases from the code, directly From IEx. While pry is running, the code execution stops, until `continue` is called. Remember you can always run `iex` in the context of a project with `iex -S mix TASK`.

Unfortunately, similar to `IO.inspect/2`, `IEx.pry/0` also requires us to change the code we intend to debug. Luckily IEx also provides a `break!/2` function which allows you set and manage breakpoints on any Elixir code without modifying its source. Brief example:

![break](pix/break.png)

Similar to `IEx.pry/0`, once a breakpoint is reached code execution stops until `continue` is invoked. However, `break!/2` does not have access to aliases and imports from the debugged code as it works on the compiled artifact rather than on source.

### Debugger

For those who enjoy breakpoints but are rather interested in a visual debugger, Erlang/OTP ships with a graphical debugger conveniently named `:debugger`. 

__TODO: learn how to invoke debugger correctly.__ Online guide suggests using `$iex -S mix` at command line.

In [3]:
defmodule DebugExample do
  def doublesum(x,y) do
    hardwork(x,y)
    end
    
  defp hardwork(x,y) do
    x = 2*x
    y = 2*y
    x+y
    end
    end

{:module, DebugExample, <<70, 79, 82, 49, 0, 0, 4, 228, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 149, 0, 0, 0, 16, 19, 69, 108, 105, 120, 105, 114, 46, 68, 101, 98, 117, 103, 69, 120, 97, 109, 112, 108, 101, 8, 95, 95, ...>>, {:hardwork, 2}}

When you start the debugger, a GUI will open on your machine.
 
![debugger](pix/debugger-elixir.gif)

### Observer

For debugging complex systems, jumping at the code is not enough. It is necessary to have an understanding of the whole virtual machine, processes, applications, as well as set up tracing mechanisms. Erlang provides `:observer` to do this.

__TODO: learn how to invoke observer correctly.__ Online guide suggests using `$iex -S mix` at command line.

`:observer.start()` opens a navigation GUI.

![observer](pix/kv-observer.png)

### Other tools

* Erlang includes a `:crashdump_viewer` to view crash dumps.
* Integration with OS level tracers, such as Linux Trace Toolkit, DTRACE, and SystemTap
* [Microstate accounting](http://erlang.org/doc/man/msacc.html) benchmarks how much time the runtime spends in low-level tasks in a short time interval.
* Mix includes a profile namespace with tools such as `cprof` and `fprof`.

The community has also created its own tools:

* [wObserver](https://github.com/shinyscorpion/wObserver) observes production nodes through a web interface.
* [visualixir](https://github.com/koudelka/visualixir) is a development-time process message visualizer.
* [erlyberly](https://github.com/andytill/erlyberly) is a GUI for tracing during development.